# Fianl Stage: Hybrid Link Prediction

## Construct the graph

### import & input

In [1]:
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import networkx as nx
import csv
import operator

In [2]:
from sklearn import cross_validation
from random import choice

/home/luy0a/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import random

In [4]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

Hello, TensorFlow!


## Preprocessing

In [5]:
f_index = 'full_paper_index.txt'
f_links = 'full_paper_reference_links.txt'
d_index = pd.read_csv(f_index, sep='\t')
d_links = pd.read_csv(f_links, sep='\t')

In [6]:
d_index['title'][0]

'Complete Mining of Frequent Patterns from Graphs: Mining Graph Data'

Seems that some source papers appeared in reference_links are not in the index file. So use papers in index file to construct the mapper and consider them as nodes during computation. 

In [7]:
def construct_mapper(d, string):
    mapper={}
    for v in range(d.shape[0]):
        mapper.setdefault(v, d[string][v])
    print(v)
    return mapper

In [8]:
p_mapper = construct_mapper(d_index, 'paper_id')
a_mapper = construct_mapper(d_index, 'authors')
t_mapper = construct_mapper(d_index, 'title')

125916
125916
125916


In [9]:
def construct_mapper_r(d, string):
    mapper={}
    for v in range(d.shape[0]):
        mapper.setdefault(d[string][v], v)
    print(v)
    return mapper

In [10]:
p_mapper_r = construct_mapper_r(d_index, 'paper_id')
a_mapper_r = construct_mapper_r(d_index, 'authors')
t_mapper_r = construct_mapper_r(d_index, 'title')

125916
125916
125916


### Create reference graph using integers rather than indices in d_links

In [11]:
f = open('ref_graph.txt', 'w')
wr = csv.writer(f, delimiter='\t', quoting=csv.QUOTE_NONE)
for i in range(d_links.shape[0]):
    source = p_mapper_r.get(d_links['source'][i], -1)
    target = p_mapper_r.get(d_links['target'][i], -1)
    if (source != -1) and (target != -1):
        wr.writerow( (source, target) )
f.close()
ref_graph = np.loadtxt('ref_graph.txt', dtype='int', delimiter='\t')

## Construct the graphs
- Now we only need to use author_mapper2.txt, paper_mapper2.txt as mappers. 
- f_coauthor_links.txt and f_paper_reference_links.txt as edges. 

In [12]:
G_paper = nx.Graph()

In [13]:
G_paper.add_nodes_from(p_mapper.keys())

In [14]:
G_paper.add_edges_from(ref_graph)

## link predictions

In [ ]:
component_list = sorted(nx.connected_component_subgraphs(G_paper), key=len, reverse = True)
Gmax = component_list[0]

In [26]:
Gmax = nx.DiGraph()
N = np.loadtxt('p_nodes')
Gmax.add_nodes_from(N)
E = np.loadtxt('p_edges')
Gmax.add_edges_from(E)

In [27]:
# Randomly mask 10% of the links
num_of_edges = len(Gmax.edges())
testIndices = np.array(random.sample(range(num_of_edges), int(0.01*num_of_edges)))
trainIndices = np.setdiff1d(range(num_of_edges), testIndices)

In [28]:
G_paper_train = nx.Graph()
G_paper_train.add_nodes_from(Gmax.nodes())
G_paper_train.add_edges_from(np.array(Gmax.edges())[trainIndices])

In [29]:
print num_of_edges
n = len(Gmax.edges())-len(G_paper_train.edges())
print len(G_paper_train.edges())
print n

140101
138499
1602


### Unsupervised: Jaccard, adamic/Adar

In [30]:
len(G_paper_train.nodes())

66185

In [31]:
import heapq
def jaccard_sim(G, N):
    score = list(nx.jaccard_coefficient(G))
    ranked_score = heapq.nlargest(N, score, key = operator.itemgetter(2))
    return ranked_score

In [32]:
def adamic_adar_sim(G, N):
    score = list(nx.adamic_adar_index(G))
    ranked_score = heapq.nlargest(N, score, key = operator.itemgetter(2))
    return ranked_score

In [33]:
jac_result = jaccard_sim(G_paper_train, n)
ada_result = adamic_adar_sim(G_paper_train, n)

KeyboardInterrupt: 

In [ ]:
def evaluate_prediction(sim, N, mapper):
    